In [ ]:
import pandas as pd
import numpy as np
import customtkinter as ctk

In [ ]:
'''
VALUES TO ASK ABOUT
in flow boundary conditions, the only if bc=62 - is this handled by another file, or do I need to handle it, what is the 0.03
On the first line of free surface parameters, what parameter is the 0.65
'''

In [ ]:
'''
#TODO: Repeat fractions n times for user input in LPT file
Create logic to only do LPT file if it is called in control
make dictionaries potentially for each input in control and LPT file so strings become values
Ask about extra 'strange' extra values
Create logic to control for non-whole numbers, iteratively adjust mesh size/block num to allow whole integers
Also create logic to make sure cells per block are not more than 100

tkinter seems like a good option. Things to do:
get inputs
    - add logic so inputs can't be certain values (mostly 0s or negatives)
    - raise value error if no value for required fields
    - add logic so certain questions show only based on user input is positive
fill forms in the tkinter way
    - add dictionaries for each string to integer value
submit button - process inputs (create dfs and then files) when user clicks submit
further:
add back button
add advanced options with toggle

Step 1 for today:
explore advanced options
handle logic for submit button
'''

In [ ]:
# Initialize the CustomTkinter app
ctk.set_appearance_mode("dark")  # Options: "light", "dark", "system"
ctk.set_default_color_theme("green")  # Options: "blue", "green", "dark-blue"

root = ctk.CTk()
root.title("CustomTkinter Scrollable Form Example")
root.geometry("500x400")

# Create a scrollable frame
scrollable_frame = ctk.CTkScrollableFrame(root, width=480, height=360)
scrollable_frame.pack(pady=10, padx=10, fill="both", expand=True)

# Add input fields to the scrollable frame
ctk.CTkLabel(scrollable_frame, text="Step 1: Control File", font=("Arial", 16)).pack(pady=10,anchor="w")

# List of sections, each containing questions
sections = [
    {
        "section_name": "Numeric Parameters",
        "questions": [
            ("What kind of scenario do you want to simulate?", "dropdown", ["channel", "cavity", "column"]),
            ("What is the ubulk value?", "float"),
            ("What is the mesh size?", "float"),
            ("What will be the main fluid, water or air?", "radio", ["water", "air"]),
            ("What discretisation scheme do you choose?","dropdown",["2nd order CDS", "4th order CDS","WENO?"]),
            ("Which timestep do you want?","float"),
            ("How many timesteps do you want to run?","int"),
            ("Do you want a restart?","radio",["yes","no"]),
            ("Do you want to calculate statistics from scratch?","radio",["yes","no"]),
            ("How often do you want to produce outputs (tecbins)?","int"),
            ("Do you want to produce transient outputs?","radio",["yes","no"]),
            ("How often do you want to produce transient outputs? (If none, enter 0)","int")
        ]
    },
    {
        "section_name": "Flow Boundary Conditions",
        "questions": [
            ("Which west boundary condition option do you want?", "dropdown", ["inflow","slip", "no-slip", 
                                "smooth log law","1/7th power law", "prescribed inflow", "SEM", 
                                "periodic condition", "rough log law", "1/6th law", "1/7th law","1/8th law"]),
            ("Which east boundary condition option do you want?", "dropdown", ["outflow", "convective", "periodic", "neumann",
                                "slip", "no-slip","smooth log law", "rough log law", "1/6th law", "1/7th law", "1/8th law"]),
            ("Which south boundary condition option do you want?", "dropdown", ["slip", "no-slip", "periodic", "smooth log law", 
                                "rough log law", "1/6th law", "1/7th law", "1/8th law"]),
            ("Which north boundary condition option do you want?", "dropdown", ["slip", "no-slip", "periodic", "smooth log law", 
                                "rough log law", "1/6th law", "1/7th law", "1/8th law"]),
            ("Which bottom boundary condition option do you want?", "dropdown", ["slip", "no-slip", "periodic", "smooth log law", 
                                "rough log law", "1/6th law", "1/7th law", "1/8th law"]),
            ("Which top boundary condition option do you want?", "dropdown", ["slip", "no-slip", "periodic", "smooth log law", 
                                "rough log law", "1/6th law", "1/7th law", "1/8th law"])
        ]
    },
    {
        "section_name": "Modelling Options",
        "questions": [
            ("At which ctime do you want to start collecting statistics?", "float"), 
            ("Do you want an SGS model?", "radio", ["yes", "no"]),
            ("Which SGS model do you want?", "dropdown", ["None","Smagorinsky","WALE","OneEqnModel","k-eps model (RANS)"]),
            ("Do you want immersed boundaries to be on or off?", "radio", ["on", "off"]),
            ("Do you want energy equation?", "radio", ["yes", "no"]),
            ("Do you want to create lagrangian particles?", "radio", ["yes", "no"]),
            ("Passive scalar?", "radio", ["yes", "no"]),
            ("Active scalar?","radio",["yes","no"])
        ]
    }
]

advanced_sections = [
    {
        "section_name": "Numeric Parameters",
        "questions": [
            ("pr:", "float"),
            ("turb_schmidt:", "float"),
            ("beta:", "float"),
            ("gx:", "float"),
            ("gy:", "float"),
            ("gz:", "float"),
            ("convection scheme:", "dropdown",["Exp.Euler","AdamsBashfort","RK2","RK3"]),
            ("diffusion scheme:", "dropdown",["Imp.Euler","CrankNicholson","ExplicitRungeKuttaDiff"]),
            ("solver:", "dropdown",["sip","mg"]),
            ("multigrid step (ngrid):", "int"),
            ("multigrid iteration scheme:", "dropdown",["GSS","TDMA","RBGSS"]),
            ("multigrid maximum iteration per timestep:", "int"),
            ("restriction iter:", "int"),
            ("prolongation iter:", "int"),
            ("variable or fixed:", "radio",["variable","fixed"]),
            ("sweeps:", "int"),
            ("safety factor:", "float"),
            ("niter:", "int"),
            ("eps:", "float"),
            ("nswp(1):", "int"),
            ("nswp(2):", "int"),
            ("nswp(3)", "int"),
            ("nswp(4)", "int"),
        ]
    },
    {
        "section_name": "Flow Boundary Conditions",
        "questions": [
            ("Friction coefficient:", "radio", ["manning", "equivalent sand"]),
            ("What is the value of the friction coefficient?:", "float"),
            ("Save inflow data (precursor sim.):", "radio",["True","False"]),
            ("Number of inlets:", "int")
        ]
    },
    {
        "section_name": "Synthetic Eddy Method",
        "questions": [
            ("Velocity Profile:", "radio", ["Uniform", "1/7th PL"]),
            ("Turbulence intensity:", "float"),
            ("Number inlet profiles", "int")
        ]
    },
    {
        "section_name": "Modelling Options",
        "questions": [
            ("time_averaging:", "radio", ["True", "False"]),
            ("noise:", "float"),
            ("LMR:", "radio",["old ghost cell approach","new ghost cell approach"]),
            ("normal_ghost_velocity_interpolation", "radio", ["2nd order", "4th order"])
            ("lrough:", "radio",["True","False"]),
            ("OpenMP threads:", "int"),
            ("pl_ex (# of extra ghost planes. pl_ex=0 -> blocks have only 1 ghost layer):", "int"),
            ("Th:", "int"),
            ("Tc:", "int"),
        ]
    },
    {
        "section_name": "Energy Boundary Conditions",
        "questions": [
            ("Do you want to adjust turbulence intensity?", "radio", ["yes", "no"]),
            ("Set custom inflow velocity profile (optional):", "float"),
            ("Set mesh refinement level:", "int"),
            ("Enable LES-specific diagnostics?", "radio", ["yes", "no"])
        ]
    },
    {
        "section_name": "Free Surface Parameters",
        "questions": [
            ("Do you want to adjust turbulence intensity?", "radio", ["yes", "no"]),
            ("Set custom inflow velocity profile (optional):", "float"),
            ("Set mesh refinement level:", "int"),
            ("Enable LES-specific diagnostics?", "radio", ["yes", "no"])
        ]
    },
    {
        "section_name": "Time Series",
        "questions": [
            ("Do you want to adjust turbulence intensity?", "radio", ["yes", "no"]),
            ("Set custom inflow velocity profile (optional):", "float"),
            ("Set mesh refinement level:", "int"),
            ("Enable LES-specific diagnostics?", "radio", ["yes", "no"])
        ]
    },
    
]

DEFAULT_VALUES = {
    #numeric parameter defaults
    "pr" : 0.72,
    "turb_schmidt": 0.6,
    "beta": 0.0034,
    "gx": 0,
    "gy": 0,
    "gz": -9.81,
    "convection_scheme": 3,
    "diffusion_scheme": 3,
    "solver": 2,
    "multigrid_step": 1,
    "multigrid_iteration_scheme": 2,
    "multigrid_max_iter": 30,
    "restriction_iter": 2,
    "prolongation_iter": 1,
    "variable_fixed": "F",
    "sweeps": 25,
    "safety_factor": 0.2,
    "niter": 20,
    "eps": 1.0E-5,
    "nswp_1": 5,
    "nswp_2": 5,
    "nswp_3": 5,
    "nswp_4": 20,
    #flow boundary defaults
    "friction_coeff": 'n',
    "only_if_bc": 0.03,
    "save_inflow_data": "F",
    "inlet_num": 5000,
    #synthetic eddy method defaults
    "velocity_profile": 12,
    "turbulence_intensity": 0.1,
    "inlet_profile_num": 1000,
    #modelling options defaults
    "time_averaging": "T",
    #"t_start_averaging_2": t_start_averaging_1 + 1,
    "noise": 0.0,
    "LMR": 2,
    "normal_ghost_velocity_interpolation": 2,
    "lrough": "F",
    "openmp_threads": 1,
    "pl_ex": 2,
    "th": 293,
    "tc": 293,
    #energy boundary conditions defaults
    "west_temp_boundary": 7,
    "east_temp_boundary": 7,
    "south_temp_boundary": 7,
    "north_temp_boundary": 7,
    "bottom_temp_boundary": 8,
    "top_temp_boundary": 7,
    #free surface parameters defaults
    "l_lsm": "F",
    "reinit": "F",
    "ntime_reinit": 3,
    "reldif_LSM": 0.001,
    "length": 0.02,
    "accuracy": 22,
    #TODO: ask about this value
    "missing_value": 0.65,
    "lends": "F",
    "L_LSMbase": "F",
    "L_LSMinit": "F",
    "L_anim_phi": "F",
    "L_anim_grid": "F",
    "densl": 1000.0,
    "densg": 1.205,
    "nul": 1.0E-6,
    "nug": 1.5E-5,
    "gx": 0.0,
    "gy": 0.0,
    "gz": -9.8,
    "slope": 0.00,
    #time series defaults
    "num_time_series_points": 1,
    "point_num_1": 1,
    "point_num_1_2": 10,
    "point_num_1_3": 10,
    "point_num_1_4": 10
    
}

# Dictionary to store answers
entries = {}
transient_var = ctk.StringVar()  # Store transient output choice
transient_frequency_var = None  # Will store transient frequency if applicable


# Function to handle creating a section with its header and questions
def create_section(section, parent_frame):
    # Create section header
    ctk.CTkLabel(parent_frame, text=section["section_name"], font=("Arial", 18, "bold")).pack(pady=10, anchor="w")

    # Create questions within this section
    for question in section["questions"]:
        question_text = question[0]
        input_type = question[1]
        create_question(question_text, input_type, question[2] if len(question) > 2 else None, parent_frame)


# Function to handle creating a question of any type
def create_question(question_text, input_type, options, parent_frame):
    ctk.CTkLabel(parent_frame, text=question_text, anchor="w").pack(pady=5, anchor="w")

    if input_type == "dropdown":
        var = ctk.StringVar(value=options[0])
        dropdown = ctk.CTkOptionMenu(parent_frame, variable=var, values=options)
        dropdown.pack(pady=5, anchor="w")
        entries[question_text] = var

    elif input_type == "radio":
        var = ctk.StringVar(value=options[0])
        radio_button_1 = ctk.CTkRadioButton(parent_frame, text=options[0], variable=var, value=options[0])
        radio_button_2 = ctk.CTkRadioButton(parent_frame, text=options[1], variable=var, value=options[1])
        radio_button_1.pack(pady=5, anchor="w")
        radio_button_2.pack(pady=5, anchor="w")
        entries[question_text] = var

    elif input_type == "radio" and question_text == "Do you want to produce transient outputs?":
        radio_button_1 = ctk.CTkRadioButton(parent_frame, text="yes", variable=transient_var, value="yes")
        radio_button_2 = ctk.CTkRadioButton(parent_frame, text="no", variable=transient_var, value="no")
        radio_button_1.pack(pady=5, anchor="w")
        radio_button_2.pack(pady=5, anchor="w")
        entries[question_text] = transient_var

    elif input_type == "float" or input_type == "int":
        entry = ctk.CTkEntry(parent_frame)
        entry.pack(pady=5, anchor="w")
        entries[question_text] = entry


# Iterate through the sections and create them
for section in sections:
    create_section(section, scrollable_frame)

# Function to process inputs when the user clicks Submit
def submit_inputs():
    print("Processing inputs...")

    for question, entry in entries.items():
        if isinstance(entry, ctk.StringVar):
            print(f"{question}: {entry.get()}")
        else:
            print(f"{question}: {entry.get()}")

    print("Finished processing inputs.")


# Add a Submit button at the bottom
submit_button = ctk.CTkButton(scrollable_frame, text="Submit", command=submit_inputs)
submit_button.pack(pady=10)

# Start the main loop
root.mainloop()

'''
add if logic
add advanced options - set to defaults if not
add logic so inputs are all required
all logic so floats and ints meet requirements
'''